In [2]:
import pybamm

parameter_set_bpx = "inital_ecm.json"
parameter_values = pybamm.ParameterValues.create_from_bpx(parameter_set_bpx)



ValidationError: 23 validation errors for BPX
Header
  field required (type=value_error.missing)
Parameterisation
  field required (type=value_error.missing)
Ambient temperature [K]
  extra fields not permitted (type=value_error.extra)
C1 [F]
  extra fields not permitted (type=value_error.extra)
C2 [F]
  extra fields not permitted (type=value_error.extra)
Cell capacity [A.h]
  extra fields not permitted (type=value_error.extra)
Cell thermal mass [J/K]
  extra fields not permitted (type=value_error.extra)
Cell-jig heat transfer coefficient [W/K]
  extra fields not permitted (type=value_error.extra)
Current function [A]
  extra fields not permitted (type=value_error.extra)
Element-1 initial overpotential [V]
  extra fields not permitted (type=value_error.extra)
Element-2 initial overpotential [V]
  extra fields not permitted (type=value_error.extra)
Entropic change [V/K]
  extra fields not permitted (type=value_error.extra)
Initial SoC
  extra fields not permitted (type=value_error.extra

In [115]:
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt

# Load your data (you've already done this part)
path = os.getcwd()
ocv_data = pybamm.parameters.process_1D_data("parameterization\\data\\G1_ocv.csv", path=path)
r0_data = pybamm.parameters.process_2D_data_csv("parameterization\\data\\r0.csv", path=path)
r1_data = pybamm.parameters.process_2D_data_csv("parameterization\\data\\r1.csv", path=path)
c1_data = pybamm.parameters.process_2D_data_csv("parameterization\\data\\c1.csv", path=path)

def ocv(sto):
    name, (x, y) = ocv_data
    result = pybamm.Interpolant(x, y, sto, name)
    return result

def r0(current, soc, *args):
    name, (x, y) = r0_data
    result = pybamm.Interpolant(x, y, [current, soc], name)
    return result

def r1(current, soc, *args):
    name, (x, y) = r1_data
    result = pybamm.Interpolant(x, y, [current, soc], name)
    return result

def c1(current, soc, *args):
    name, (x, y) = c1_data
    result = pybamm.Interpolant(x, y, [current, soc], name)
    print(f"c1({current}, {soc}) = {result}")
    return result


model = pybamm.equivalent_circuit.Thevenin()

print(model.variable_names())

parameter_values = pybamm.ParameterValues("ECM_Example")
parameter_values.update({
    "R0 [Ohm]": r0,
    "R1 [Ohm]": r1,
    "C1 [F]": c1,
})

experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/10 for 10 hours or until 3.3 V",
        ),
    ]
)
solver = pybamm.CasadiSolver()

sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment, solver=solver)
sim.solve()
sim.plot()

The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
The right-hand side routine failed at the first call.
2025-03-12 21:10:03.824 - [ERROR] callbacks.on_experiment_error(233): Simulation error: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:1432:
Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:361:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_FIRST_RHSFUNC_ERR". Consult CVODES documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up

['Time [s]', 'Time [min]', 'Time [h]', 'Current variable [A]', 'Total current density [A.m-2]', 'Current [A]', 'C-rate', 'SoC', 'Open-circuit voltage [V]', 'Element-1 overpotential [V]', 'Cell temperature [degC]', 'Cell temperature [K]', 'Jig temperature [degC]', 'Jig temperature [K]', 'Ambient temperature [degC]', 'Ambient temperature [K]', 'Heat transfer from cell to jig [W]', 'Heat transfer from jig to ambient [W]', 'Entropic change [V/K]', 'Reversible heat generation [W]', 'R0 [Ohm]', 'Element-0 overpotential [V]', 'Element-0 irreversible heat generation [W]', 'R1 [Ohm]', 'C1 [F]', 'tau1 [s]', 'Element-1 irreversible heat generation [W]', 'Distributed SoC', 'x ECMD', 'Diffusion overpotential [V]', 'Surface SoC', 'Irreversible heat generation [W]', 'Total heat generation [W]', 'Voltage [V]', 'Overpotential [V]', 'Battery voltage [V]', 'Power [W]', 'Resistance [Ohm]']
c1(Cell temperature [degC], 10.0) = zation\data\c(Cell temperature [degC], 10.0)


SolverError: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:1432:
Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:361:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_FIRST_RHSFUNC_ERR". Consult CVODES documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.

In [42]:
#model = pybamm.equivalent_circuit.Thevenin()
model = pybamm.lithium_ion.SplitOCVR()

parameter_values = pybamm.ParameterValues(values=get_parameter_values())

t_eval = [0,3600]
solver = pybamm.CasadiSolver(mode="fast", atol=1e-6, rtol=1e-3)

sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment, solver=solver)
sim.solve(t_eval=t_eval)
sim.plot()

SolverError: Events ['Minimum voltage [V]'] are non-positive at initial conditions

In [39]:
print("R0 Data:", r0_data)

R0 Data: ('parameterization\\data\\r0', ((array([298.15]), array([2.6603632 , 2.66038561, 2.66040325, 2.66042423, 2.66043043,
       2.66043234, 2.6604352 , 2.66044092, 2.66046524, 4.85023355])), array([[99.95057391, 92.3934429 , 81.33499995, 71.35668487, 61.46862011,
        50.7033229 , 42.65453546, 33.24365645, 22.33075269, 12.40409754]])))
